In [ ]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pointcloud, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[0.2, 0.2, 0.2],
                                  up=[0.2304, -0.8825, 0.4101])

In [ ]:
def extract_geodesic_patch(coords, normals, center_index, radius):

    cluster_point_idx = [center_index] # to save all the points that are within the geodesic radius
    
    #loop through all the points and check what their distance to the selected center point is
    for index, point in enumerate(coords):
        dist = np.linalg.norm(coords[center_index]-point)

        # select only those points that are close to the center point and calculate their geodesic distance
        if dist < radius and dist != 0:
            #dij = dist+((1-np.dot(normals[index], center_norm))*dist)
            dij = dist*(2-np.dot(normals[index], center_norm))# approximation of geodesic distance

            # select only those points that are *geodesically* close to the center point
            if dij < radius:
                cluster_point_idx.append(index)
    
    return cluster_point_idx

In [ ]:
def extract_geodesic_patch(coords, normals, center_index, radius):
    
    first_sel = [] #save indeces of first selection
    cluster = [center_index] #save indeces of cluster-members
    geod_distances_from_center = {center_index:0} #save geodesic distances to the center of cluster-members
    
    #loop through all the points and check what their distance to the selected center point is < radius
    for index, point in enumerate(coords):
        dist = np.linalg.norm(coords[center_index]-point)

        # select only those points that are close to the center point (non-geodesically)
        if dist < radius and dist != 0:
            first_sel.append(index)

            
    while len(first_sel) != 0:
        
        # find the point in first_sel, which is closest to any point in the cluster
        shortest_dist = 10^6
        closest_in_sel = 10^6
        closest_in_cluster = 10^6

        # compare each element in first_sel with each element in cluster and approximate their geodesic distance
        for sel in first_sel:
            for point in cluster:
                geo_dist = np.linalg.norm(coords[sel]-coords[point])*(2-np.dot(normals[sel], normals[point]))
                if geo_dist < shortest_dist:
                    closest_in_sel = sel
                    closest_in_cluster = point
                    shortest_dist = geo_dist

        # check if the found closest point should be part of the cluster
        dist_from_center = geod_distances_from_center[closest_in_cluster] + shortest_dist
        if dist_from_center < radius:
            # if yes, append its index to cluster, save its geod_dist_from_center and remove it from first_sel
            cluster.append(closest_in_sel)
            geod_distances_from_center[closest_in_sel]=dist_from_center
            first_sel.remove(closest_in_sel)
        else:
            first_sel.remove(closest_in_sel)
    
    return cluster     